**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

---

Mentoría #13 - Cómo hacer un Clasificador de Pliegos todoterreno (y de otros tipos de textos) usando NLP

---
Integrantes Grupo [1|2]:
*   Bruce Wayne: bruce.w@wayne-entreprises.com
*   Alfred Pennyworth: alfred@msn.com
*   Richard Grayson: robin_132@gmail.com
---

Edición 2023


# Introducción

En esta Mentoría, trabajaremos con un conjunto de datos que comprende aproximadamente 100.000 pliegos y licitaciones de diversos organismos nacionales, tanto públicos como privados. Estos datos se obtuvieron de un sistema/servicio diseñado para monitorear oportunidades de negocios, capturar la información en una base de datos, normalizarla y, posteriormente, clasificarla para informar a los usuarios según sus áreas de interés. Los usuarios de este sistema reciben alertas automáticas cada vez que se publica una oportunidad comercial que coincide con su perfil.

## Desafío

Como parte del proceso de clasificación, los pliegos se etiquetan utilizando principalmente reglas estáticas, como palabras clave, lo cual deja margen para optimizaciones. El título de cada pliego y principalmente la descripción de los objetos que se están licitando son campos de tipo texto y escritos por personas, por lo que naturalmente presentan ambigüedades y características propias del lenguaje que llevan a que un enfoque rígido y estático de clasificación, como el actual, resulte limitado y poco eficiente.

El Desafío es utilizar las técnicas de Aprendizaje Automático, logrando así un "Clasificador" que utilice técnicas de Procesamiento de Lenguaje Natural (NLP) para clasificar de manera más eficiente en qué rubro o categoría se encuentra un pliego, basándose en su texto descriptivo y otros campos relacionados.

## Interés General

Más allá de la aplicación específica en este conjunto de datos, este problema, al estar vinculado al Procesamiento de Lenguaje Natural (PLN) y la clasificación, tiene la ventaja de poder ser utilizado posteriormente para otros tipos de contenido. De esta manera, el clasificador desarrollado podría aplicarse para categorizar libros, noticias, textos, tweets, publicaciones, artículos, etc.

## Descripción del dataset

A continuación se enumeran las diferentes variables del dataset, así como una breve descripción de su significado:

- **id**: Clave única y primaria autoincremental de la tabla;
- **cargado**: Fecha de carga del pliego;
- **idexterno**: Id del pliego en la fuente;
- **referencia**: Campo auxiliar obtenido de la fuente;
- **objeto**: Campo principal, descripción del producto o servicio objeto de la licitación;
- **rubro**: Campo de categorización disponible en la fuente. No siempre está disponible;
- **agencia**: Empresa o Ente que lanza la licitación;
- **apertura**: Fecha de apertura del pliego (vencimiento para presentarse al pliego);
- **subrubro**: Subcategoría del pliego (también obtenido desde la fuente);
- **pais**: País donde se lanza la licitación;
- **observaciones**: Campo auxiliar donde se guardan datos extra que puede variar según la fuente;
- **monto**: Monto del pliego, no siempre está publicado;
- **divisaSimboloISO**: Moneda en la que se especifica el pliego;
- **visible**: campo binario que determina si el pliego va a ser visualizado por el sistema (True/False);
- **categoría**: Tipo de pliego, categorización entre diversos tipos de pliegos (Compra Directa, Licitación Simple, Subasta, etc.);
- **fuente**: Fuente de donde se obtuvo la licitación.

# Trabajo Práctico #2

Como vimos en el primer Trabajo Práctico, en esta segunda entrega tenemos como objetivo preparar los datos para poder alimentar un modelo de predicción. Aplicaremos diferentes tipos de prácticas y estrategias para preparar el dataset y resolver todas las cuestiones detectadas en el TP1

A continuación se enumeran los ejercicios propuestos como guía:

## Ejercicio 1: Limpieza de Datos

Según lo vimos en el TP1, aplicar las técnicas propuesta para limpiar los datos. En todos los casos, además de aplicar la técnica, redactar brevemente la explicación del criterio tomado.

1.   Selección de Variables. Como vimos, hay variables que pueden descartarse.
2.   Corrección de los diferentes Tipos de Variables. Aplicar las conversiones de tipo necesarias.
3.   Selección de Registros. Filtrar las filas que no utilizaremos para la clasificación.
4.   Curación de la variable [monto]. Tip: usar la variable [divisaSimboloISO]
5.   Curación de la variable [fecha]. Incluye la eliminación de Outliers.
6.   En [agencia], normalizar si es que existen entradas con valores equivalentes.
7.   (Opcional) Para las variables categóricas aplicar técnicas de encoding, de manera de ir transformando variables de texto en números que los algoritmos puedan procesar.

## Ejercicio 2: Variable [rubro]

1.   Unificar categorías (rubros) equivalentes entre distintas fuentes. Usar un criterio definido por el grupo y explicarlo en pocas palabras.
2.   Utilizar visualizaciones que permitan ver todas las categorías y verificar que sean valores válidos.
3.   En caso de haber registros con categorías inválidas, tomar algún criterio para imputar la categoría o eliminar el registro. Desarrollar la decisión tomada.
4.   Resolver si se va a utilizar subrubro para aperturar la categorización en algún caso. Definir luego qué postura tomar con respecto a la variable [rubro]
5.   En las casos en que existe más de una categoría, tomar alguna postura para que los registros finalmente queden con una única etiqueta.

## Ejercicio 3: Variable [objeto]

1.   Aplicar técnicas de lematización para eliminar artículos, conectores, stop_words en general. Quitar acentos (tildes), mayúsculas, plurales, caractéres inválidos, abreviaturas, etc., dejando solo palabras importantes para la clasificación.
2.   Utilizar técnicas de visualización para detectar otros términos muy frecuentes que no aportan valor a la clasificación.
3.   (Opcional) Aplicar técnicas de tokenización para convertir la variable objeto en un array de tokens.

## Ejercicio 4: Exportación

Con el dataset ya transformado, generar una versión final y exportarla a un nuevo archivo. Este es el resultado final del TP2.

## Ejercicio 5: Análisis

En base a todo lo aprendido en las materias obligatorias cursadas, qué tipo de modelos de clasificación piensan que podrían aplicarse al dataset obtenido? Desarrollar brevemente una justificación en comparación con otros modelos posibles.

# Desarrollo

# Librerías

Importamos librerías necesarias para la visualización, y para desactivar warnings.

In [ ]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  # default='warn'

#levantamos el dataset
df = pd.read_csv("./datos/Mentoria_Dataset_0.csv" , encoding='latin-1')

# Filtrado

Primero, hacemos la limpieza del dataset de acuerdo a lo que vimos en el TP1.

Empezamos por quedarlos con los datos que tienen *categoría*=1, y las variables *referencia*, *objeto*, *rubro*, *agencia*, *apertura*, *monto* y *fuente*.

In [ ]:
#tiro los que tienen categoria 2
df_fil = df[df['categoria']==1]
#variables que decidimos relevantes
variables = ['referencia', 'objeto', 'rubro','agencia','apertura','monto','fuente']
df_fil = df_fil[variables]

# Limpieza de las fechas

Ahora hacemos la limpieza de las fechas que no tienen sentido:

In [ ]:
#Hacemos el filtrado de fechas hecho en el práctico anterior:

today_date = pd.Timestamp.today().date()
min_date = pd.to_datetime('13/12/2015')
df_fil.reset_index(inplace=True)  #reseteo indices, es util cuando indice=nro de fila

#fechas malas que encontramos:
fecha_mala1 = df_fil['apertura'][62]
fecha_mala2 = df_fil['apertura'][2025]
fecha_mala3 = df_fil['apertura'][11907]

#removemos las fechas malas
df_fil = df_fil.drop(df_fil[df_fil['apertura'] == fecha_mala1].index)
df_fil = df_fil.drop(df_fil[df_fil['apertura'] == fecha_mala2].index)
df_fil = df_fil.drop(df_fil[df_fil['apertura'] == fecha_mala3].index)

#convierto a formato datetime
df_fil['apertura'] = pd.to_datetime(df_fil['apertura'],format='%d/%m/%Y %H:%M')
df_fil['apertura'] = df_fil['apertura'].dt.tz_localize(None)
df_fil['apertura'] = df_fil['apertura'].dt.date

#removemos outliers
df_fil = df_fil.drop(df_fil[df_fil['apertura'] > today_date].index)
df_fil = df_fil.drop(df_fil[pd.to_datetime(df_fil['apertura']) < min_date].index)


# Limpieza de la variable monto

Limpiamos ahora la variable monto. En las entradas donde han NaN's, las definimos como 0. Además, convertimos dólares a peso y cambiamos strings a valores numéricos.

In [ ]:
#precio del dolar oficial
dolar_value = 350

df_fil['monto'] = df_fil['monto'].replace(r'\N', np.nan)
df_fil['monto'] = df_fil['monto'].replace('0', 0)
df_fil['monto'] = df_fil['monto'].replace('NaN', 0)

#remuevo las strings de espacios, pesos y dolares donde las haya. Cambio la representacion de string a float
j=0
for monto, i in zip(df_fil['monto'],df_fil.index):
    if " " in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace(' ','')
    if "$" in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace('$','')
    if "." in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace('.','')
    if "," in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace(',','.')
    if "U$S" in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace('US','')
        #convierto a pesos y reemplazo la entrada
        df_fil['monto'][i] = float(df_fil['monto'][i]) * dolar_value
        j+=1
    df_fil['monto'][i] = float(df_fil['monto'][i])


# Variable Objeto

Ahora pasamos a procesar la variable *objeto*.

Para empezar, importamos la librería `string` para trabajar con caracteres.
Definimos tablas para hacer traducciones de caracteres no deseados, como por ejemplo todos los signos de puntuación, y números.

No queremos que haya números en la columna *objeto* porque consideramos que tener, por ejemplo:

* direcciones
* palabras como '4x4'
* precios
* cantidad de productos o servicios

no aportan nada al desarrollo de esta variable.

Además, eliminamos tildes, pasamos todas las palabras a minúsculas y sacamos un montón de caracteres raros.

**Warning**: esta celda tarda mucho.

In [ ]:
import string

#tabla para reemplazar signos de puntuacion
table_punct = str.maketrans(' ', ' ', string.punctuation)
#tabla para reemplazar numeros
table_num = str.maketrans(' ', ' ', '1234567890')
table_otros = str.maketrans(' ', ' ', 'º°¿´ª¨®·¥½ðï¤¾µ')

#defino los datasets a limpiar
df_test = df_fil[['objeto','rubro']]
print(df_test['objeto'][0:50])

df_test.reset_index(inplace=True)

i=0
for i in range (len(df_test)):
#for i in range (100):
    df_test['objeto'][i] = str(df_test['objeto'][i]).lower()  #pasamos todo a minusculas
    df_test['objeto'][i] = str(df_test['objeto'][i]).translate(table_punct) #eliminamos signos de puntuacion
    df_test['objeto'][i] = str(df_test['objeto'][i]).translate(table_num) #eliminamos numeros
    df_test['objeto'][i] = str(df_test['objeto'][i]).translate(table_otros) #eliminamos acentos y otros

    sent = str(df_test['objeto'][i])   # variable a analizar
    if "á" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('á','a')
    if "é" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('é','e')
    if "í" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('í','i')
    if "ó" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ó','o')
    if "ú" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ú','u')
    if "à" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('à','u')
    if "è" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('è','u')
    if "ì" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ì','u')
    if "ò" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ò','u')
    if "ù" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ù','u')
    if "ü" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ü','u')
    if "ë" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ë','e')
    if "ã" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ã','a')
    if "ä" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ä','a')
    if "ø" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ø','o')
    if "ç" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ç','c')
    if "ä" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ä','a')
    if "m²" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('m²','')
    if "m³" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('m³','')
    if "û" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('û','u')
    if "ö" in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('ö','o')
    if '\x94' in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('\x94',' ')
    if '\x92' in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('\x92',' ')
    if '\x91' in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('\x91',' ')
    if '\xad' in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('\xad',' ')
    if '\x95' in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('\x95',' ')
    if '\x81' in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('\x81',' ')
    if '\x99' in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('\x99',' ')
    if '\x8d' in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('\x8d',' ')
    if '\x96' in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('\x96',' ')
    if '\x93' in sent:
        df_test['objeto'][i] = df_test['objeto'][i].replace('\x93',' ')

    i+=1

print(df_test['objeto'][0:50])


Ahora, pasamos a utilizar librerías específicas de PLN para tokenizar y lematizar las entradas del dataset.

Para eso, usamos la librería `nltk`.

`word_tokenize` es una función para convertir a una oración (una string larga) en una lista de sus palabras constituyentes. Cada entrada de dicha lista se denomina **token**.

`stopwords` es una lista de palabras conectoras que no aportan nada de información en un modelo de PLN y que sólo aumentan la dimensionalidad del problema.

`SnowballStemmer` Es la función que vamos a utilizar para convertir a cada palabra en su raíz, y así agrupamos muchas palabras que contengan la misma información (por ejemplo 'mantener' y 'mantenimiento').


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

#funcion para lematizar una palabra
def lemmatize_word(word):
    stemmer = SnowballStemmer('spanish')
    return stemmer.stem(word)


Vemos que en las 1000 primeras entradas del dataset, hay algún NaN que quedó dando vueltas que se convirtió en string. Removemos esas entradas.

In [ ]:

df_test0 = df_test.copy()

print(df_test0['objeto'][:15])
#elimino algunos nans que aparecen
df_test0 = df_test0.drop(df_test0[df_test0['objeto']=='nan'].index)
print('*'*100)
print(df_test0['objeto'][:15])


Ahora tokenizamos la variable para trabajar mejor con elementos individuales:

In [ ]:
df_test1 = df_test0.copy()

print('antes de tokenizar:')
print()
print(df_test1['objeto'][0:10])
print('-'*100)
print('después de tokenizar:')
print()
df_test1['objeto'] = df_test1['objeto'].apply(word_tokenize)
print(df_test1['objeto'][:10])


Para tener una idea de la distribución de palabras, definimos una función que devuelve una lista con la cantidad de veces que cada palabra se repite:

In [ ]:
def CountWords(df,column='objeto'):
    '''
    funcion para contar la ocurrencia de palabras en una columna de un dataset.
    La columna ya tiene que venir tokenizada
    '''

    df1 = df.copy()

    # Unimos las palabras lematizadas
    df1 = df1[column].apply(lambda words: ' '.join(words))
    entire_words = "".join(' '+item for item in df1) #el espacio ' ' es para que no pegue palabras
    list_ = entire_words.split()
    df_list = pd.DataFrame(list_,columns=['palabras'])
    counts = df_list.value_counts()

    return counts


Utilicemos esta función para ver cómo v quedando nuestro dataset:

In [ ]:
counts = CountWords(df_test1)
print(counts[:1000])


Ahora, eliminamos stopwords y palabras que se repiten bastante, que creemos que no aportan nada al modelo:

In [ ]:
df_test2 = df_test1.copy()
print('Removemos stopwords:')
print()
stop_words = set(stopwords.words('spanish'))
df_test2['objeto'] = df_test2['objeto'].apply(lambda words: [word for word in words if word not in stop_words])
print(df_test2['objeto'][:20])
print('*'*100)
print('Removemos palabras que creemos que no aportan nada al dataset:')
print()
adq_list = ['adquisicion','adq','adquis', 'adquirir', 'adquisiion',
            'contrato','contratacion', 'compra', 'provision',
            'servicio','alquiler','alq','subasta', 'locacion',
            'monto','estimado', 'renovacion']
df_test2['objeto'] = df_test2['objeto'].apply(lambda words: [word for word in words if word not in adq_list])
print(df_test2['objeto'][:20])


Vemos que la distribución de palabras se va achicando y tienden a quedar palabras importantes. Sigue habiendo cosas que no estamos seguros de remover o no, como 'n' o 'er'.

In [ ]:
counts = CountWords(df_test2)
print(counts[:100])

Finalmente, aplico la lematización para reducir las palabras a su raíz:

In [ ]:
df_test3 = df_test2.copy()
print('Aplicamos lematización a cada palabra')
print()
df_test3 = df_test3.copy()
df_test3['objeto'] = df_test3['objeto'].apply(lambda words: [lemmatize_word(word) for word in words])
print(df_test3['objeto'][:20])
print('*'*100)

Muchas de las palabras lematizadas que aparecen sólo una vez no tienen sentido, algunas porque están mal escritas, otras porque son mezclas de palabras. Vemos que casi la mitad de las palabras distintas de la variable *objeto* aparecen sólo una vez, y son representan el el 2% del dataset.

**Pregunta para el grupo:** Tiene sentido eliminar todas esas palabras?

In [ ]:
counts = CountWords(df_test3)
nwords = np.sum(counts)

print(counts[-20:])
print('*'*100)
print('nro de palabras que aparecen sólo una vez: ',len([item for item in counts if item==1]))
print('fracción del total de palabras que representan: ',len([item for item in counts if item==1])/nwords)

print('cantidad de palabras distintas:',len(counts))
print('cantidad total de palabras: ',nwords)
print('las 1000 palabras más frecuentes abarcan el '+str(round(np.sum(counts[:1000])/nwords*100,2))+'% del total')
print('las 2000 palabras más frecuentes abarcan el '+str(round(np.sum(counts[:2000])/nwords*100,2))+'% del total')
print('las 5000 palabras más frecuentes abarcan el '+str(round(np.sum(counts[:5000])/nwords*100,2))+'% del total')
print('las 10000 palabras más frecuentes abarcan el '+str(round(np.sum(counts[:10000])/nwords*100,2))+'% del total')
print('las 20000 palabras más frecuentes abarcan el '+str(round(np.sum(counts[:20000])/nwords*100,2))+'% del total')


**pregunta al grupo:** Esto no se si tiene que ir o no, no se qué les parece si remover todas esas palabras

In [ ]:
df_test4 = df_test3.copy()
df_test4.reset_index(inplace=True)

#viendo las 1000 palabras mas usadas, creemos que se pueden remover todoas estas palabras:
a_remover_lst = ['er','año','años','ordoñez','trim','do','etc','pte','sg','to','x',
'sa','grl','b','ii','nro','ra','at','usd','in','srl','expte','kg'
'iii','rn','g','v','mm','hs','m','pn','t','et','av','xii','bv','n','p']

for i in range(len(df_test4)):
    list_ = df_test4['objeto'][i]
    for word in list_:
        if word in a_remover_lst:
            df_test4['objeto'][i].remove(word)

counts2 = CountWords(df_test4)

for i in range(1000):
#    print(i,counts_lem.index[i],counts_lem.values[i])
    print(i,counts2.index[i],counts2.values[i])
#    print('-'*100)

In [ ]:
df_fil['monto'].fillna(0, inplace=True)
df_fil['monto']

In [ ]:
#df_test4.head()

df_fil[['objeto', 'rubro']] = df_test4[['objeto', 'rubro']].copy()

# Guarda el nuevo dataset en un archivo CSV o en el formato deseado
df_fil.to_csv('dataset_concatenado.csv', index=False)

df_fil.head()

# Limpieza de la variable Rubro

In [ ]:
df_fil['rubro']

In [ ]:
import string
#tabla para reemplazar signos de puntuacion
table_punct = str.maketrans(string.punctuation, ' '*len(string.punctuation))
#tabla para reemplazar numeros
table_num = str.maketrans('1234567890', ' '*len('1234567890') )
table_otros = str.maketrans('º°¿´ª¨®·¥½ðï¤¾µ', ' '*len('º°¿´ª¨®·¥½ðï¤¾µ'))

#defino los datasets a limpiar
df_test = df_fil[['objeto','rubro']].copy()

#esto es para reemplazar el encoding horrible de html
df_test.reset_index(inplace=True)
import html

for i in range (len(df_test)):
    df_test['rubro'][i] = html.unescape(str(df_test['rubro'][i]))



i=0
for i in range (len(df_test)):
#for i in range (100):
    df_test['rubro'][i] = str(df_test['rubro'][i]).lower()  #pasamos todo a minusculas
    df_test['rubro'][i] = str(df_test['rubro'][i]).translate(table_punct) #eliminamos signos de puntuacion
    df_test['rubro'][i] = str(df_test['rubro'][i]).translate(table_num) #eliminamos numeros
    df_test['rubro'][i] = str(df_test['rubro'][i]).translate(table_otros) #eliminamos acentos y otros

    sent = str(df_test['rubro'][i])   # variable a analizar
    if "á" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('á','a')
    if "é" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('é','e')
    if "í" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('í','i')
    if "ó" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('ó','o')
    if "ú" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('ú','u')
    if "à" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('à','u')
    if "è" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('è','u')
    if "ì" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('ì','u')
    if "ò" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('ò','u')
    if "ù" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('ù','u')
    if "ü" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('ü','u')
    if "ë" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('ë','e')
    if "ã" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('ã','a')
    if "ä" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('ä','a')
    if "ø" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('ø','o')
    if "ç" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('ç','c')
    if "ä" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('ä','a')
    if "m²" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('m²','')
    if "m³" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('m³','')
    if "û" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('û','u')
    if "ö" in sent:
        df_test['rubro'][i] = df_test['rubro'][i].replace('ö','o')
#     if '\x94' in sent:
#         df_test['rubro'][i] = df_test['rubro'][i].replace('\x94',' ') #sombrero
#     if '\x92' in sent:
#         df_test['rubro'][i] = df_test['rubro'][i].replace('\x92',' ') #tilde
#     if '\x91' in sent:
#         df_test['rubro'][i] = df_test['rubro'][i].replace('\x91',' ') #esto es una eñe
#     if '\xad' in sent:
#         df_test['rubro'][i] = df_test['rubro'][i].replace('\xad',' ')
#     if '\x95' in sent:
#         df_test['rubro'][i] = df_test['rubro'][i].replace('\x95',' ')
#     if '\x81' in sent:
#         df_test['rubro'][i] = df_test['rubro'][i].replace('\x81',' ')
#     if '\x99' in sent:
#         df_test['rubro'][i] = df_test['rubro'][i].replace('\x99',' ')
#     if '\x8d' in sent:
#         df_test['rubro'][i] = df_test['rubro'][i].replace('\x8d',' ')
#     if '\x96' in sent:
#         df_test['rubro'][i] = df_test['rubro'][i].replace('\x96',' ')
#     if '\x93' in sent:
#         df_test['rubro'][i] = df_test['rubro'][i].replace('\x93',' ')

    i+=1



In [ ]:
df_test0 = df_test.copy()

print(df_test0['rubro'][:1000])
#elimino algunos nans que aparecen
df_test0 = df_test0.drop(df_test0[df_test0['rubro']=='nan'].index)
print('*'*100)
print(df_test0['rubro'][:30000])

In [ ]:
df_test1 = df_test0.copy()

print('antes de tokenizar:')
print()
print(df_test1['rubro'][0:10])
print('-'*100)
print('después de tokenizar:')
print()
df_test1['rubro'] = df_test1['rubro'].apply(word_tokenize)
print(df_test1['rubro'][:30000])

In [ ]:
def CountWords(df,column='rubro'):
    '''
    funcion para contar la ocurrencia de palabras en una columna de un dataset.
    La columna ya tiene que venir tokenizada
    '''

    df1 = df.copy()

    # Unimos las palabras lematizadas
    df1 = df1[column].apply(lambda words: ' '.join(words))
    entire_words = "".join(''+item for item in df1) #el espacio ' ' es para que no pegue palabras
    list_ = entire_words.split()
    df_list = pd.DataFrame(list_,columns=['palabras'])
    counts = df_list.value_counts()

    return counts

In [ ]:
counts = CountWords(df_test1)
print(counts[:1000])


In [ ]:
df_test2 = df_test1.copy()
print('Removemos stopwords:')
print()
stop_words = set(stopwords.words('spanish'))
df_test2['rubro'] = df_test2['rubro'].apply(lambda words: [word for word in words if word not in stop_words])
print(df_test2['rubro'][:20])
print('*'*100)
print('Removemos palabras que creemos que no aportan nada al dataset:')
print()
adq_list = ['y','de','para', 'del', 'e',
            'la','yo', 'uso', 'nbsp',
            'sus','el','gral','no', 'en',
            'p','puso', 'los','x','d','eq',
            'a','he','h','o','per','tr' ,'te',
            'co','me','qu','na','l','s','in','i']
df_test2['rubro'] = df_test2['rubro'].apply(lambda words: [word for word in words if word not in adq_list])
print(df_test2['rubro'][:20])


In [ ]:
counts = CountWords(df_test2)
print(counts[:1000])

In [ ]:
df_fil['rubro'].fillna(0)

In [ ]:
df_fil[['rubro']] = df_test2[['rubro']]

# Guarda el nuevo dataset en un archivo CSV o en el formato deseado
df_fil.to_csv('dataset_concatenado.csv', index=False)

df_fil.tail()